Inference Testing v_0.1 Faretra

In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2
import pickle

In [2]:
train_path=r"C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_train"
pred_path=r"C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred"

#categories
train_root=pathlib.Path(str(train_path))
classes= os.listdir(train_root)

print("Training classes are")
print(classes)
print(pred_path)

Training classes are
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred


In [3]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1  w=Width,Heigth, f=kernelSize , P=padding , s=stride
        #Input shape= (256,3,150,150)--->(batchSize,colorChannels,Xsize,Ysize)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Output shape (256,12,150,150)-->increased channel number
        self.bn1=nn.BatchNorm2d(num_features=12)  #n_features=channels
        #Output shape (256,12,150,150)
        self.relu1=nn.ReLU()
        #Output shape (256,12,150,150)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        #Reduce the image size by factor 2
        #Output shape (256,12,75,75)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Output shape (256,20,75,75)
        self.relu2=nn.ReLU()
        #Output shape (256,20,75,75)
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Output shape (256,32,75,75)-->increased channel number
        self.bn3=nn.BatchNorm2d(num_features=32)  #n_features=channels
        #Output shape (256,32,75,75)
        self.relu3=nn.ReLU()
        #Output shape (256,32,75,75)

        
        #The syntax we saw on the previous lines is how we can add multiple convolution layers to our model.
        #By adding more steps that gradually increase the number of channels and the accuracy fo the model
        
        self.fc=nn.Linear(in_features=32*75*75,out_features=num_classes)
        
        #Feed forward Function
        

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        #Output is going to be in matrix form(256,32,75,75), 
        #to feed the output we're going to reshape it first
            
        output=output.view(-1,32*75*75)
            
        output=self.fc(output)
            
        return output
        

In [4]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [5]:
transformer= transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize ([0.5,0.5,0.5], #0-1 to [-1,1] , formula(x-mean)/std
                         [0.5,0.5,0.5])
])

In [6]:
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [7]:
images_path=glob.glob(pred_path+'/*.jpg')

print(images_path)

['C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10004.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10005.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10012.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10013.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10017.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10021.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\1003.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10034.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10038.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10040.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10043.jpg', 'C:/Users/LT_J/OneDrive/Desktop/ML/InnovatesApp/Archivio_Img/seg_pred\\10045.jpg', 'C:/

In [8]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [9]:
pred_dict

{'seg_pred\\10004.jpg': 'mountain',
 'seg_pred\\10005.jpg': 'mountain',
 'seg_pred\\10012.jpg': 'street',
 'seg_pred\\10013.jpg': 'glacier',
 'seg_pred\\10017.jpg': 'glacier',
 'seg_pred\\10021.jpg': 'forest',
 'seg_pred\\1003.jpg': 'sea',
 'seg_pred\\10034.jpg': 'glacier',
 'seg_pred\\10038.jpg': 'sea',
 'seg_pred\\10040.jpg': 'street',
 'seg_pred\\10043.jpg': 'sea',
 'seg_pred\\10045.jpg': 'street',
 'seg_pred\\10047.jpg': 'sea',
 'seg_pred\\10048.jpg': 'buildings',
 'seg_pred\\10052.jpg': 'buildings',
 'seg_pred\\10054.jpg': 'glacier',
 'seg_pred\\10059.jpg': 'forest',
 'seg_pred\\10060.jpg': 'forest',
 'seg_pred\\10066.jpg': 'mountain',
 'seg_pred\\10069.jpg': 'sea',
 'seg_pred\\10073.jpg': 'street',
 'seg_pred\\10079.jpg': 'street',
 'seg_pred\\1008.jpg': 'street',
 'seg_pred\\10082.jpg': 'street',
 'seg_pred\\10083.jpg': 'mountain',
 'seg_pred\\10090.jpg': 'glacier',
 'seg_pred\\10092.jpg': 'forest',
 'seg_pred\\10096.jpg': 'mountain',
 'seg_pred\\101.jpg': 'glacier',
 'seg_pred\